In [ ]:
# import sqlite3
# import os
# import sys
# import pickle
# import json
# import time
# import threading
# import datetime as dt
# import pandas as pd

# sys.path.append(os.path.join(os.getcwd(), '../..'))
# # from ..fitbit import Fitbit, FitbitOauth2Client, gather_keys_oauth2
# from fitbit import Fitbit, FitbitOauth2Client, gather_keys_oauth2, exceptions
# from DashBoard.src.constants import ( TOKEN_PATH,
#                             CREDS_PATH,
#                             REFRESH_TIME,
#                             DAEMON_SLEEP,
#                             RATE_LIMIT,
#                             DAYS_TO_UPDATE,
#                             NUM_REQUESTS_PER_UPDATE,
#                             SLEEP_TIME,
#                             DB_PATH)

# def sleep(s:int)->None:
#     '''
#     This is interuptable
#     '''
#     t=0
#     while t < s:
#         time.sleep(1)
#         t += 1

# class FitbitTokenDaemon(threading.Thread):
#     def __init__(self, fitbitServer: 'FitbitServer', refresh_time: int = REFRESH_TIME, percentage: float = 0.8, sleep_interval: int = DAEMON_SLEEP):
#         super().__init__()
#         self.fitbitServer = fitbitServer
#         self.refresh_time = refresh_time
#         self.percentage = percentage
#         self.sleep_interval = sleep_interval
#         self._stop_event = threading.Event()

#     def run(self):
#         while self.fitbitServer.running:
#             report = "[{}] ".format(dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
#             try:
#                 # Refresh token if it's about to expire
#                 report += "FitbitTokenDaemon 'Token expires in: {:,.0f}s'".format(self.fitbitServer.client.session.token['expires_at'] - time.time())
#                 if self.fitbitServer.client.session.token['expires_at'] - self.refresh_time * (1 - self.percentage) < time.time():
#                     report += "\n\t\tFitbitTokenDaemon 'Refreshing Token'"
#                     self.fitbitServer.client.refresh_token()
#             except Exception as e:
#                 report += f"FitbitTokenDaemon encountered an error: {e}"
#             finally:
#                 print(report)
#                 sleep(self.sleep_interval)

# class FitbitServer(Fitbit):
#     # time series data
#     BODY = ['weight','fat','bmi']
#     ACTIVITY = 'activityCalories calories caloriesBMR distance floors minutesSedentary minutesLightlyActive minutesFairlyActive minutesVeryActive steps'.split()
#     FOODS = ['caloriesIn','water']
#     TS_RESOURCE = dict(body = BODY,activities = ACTIVITY,foods = FOODS)

#     def __init__(self, expires_at=None,redirect_uri=None, system='', **kwargs):
#         self.client_id,self.client_secret = self.__get_creds()
#         token = self.__get_token()
#         super(FitbitServer,self).__init__(client_id = self.client_id,
#                                           client_secret = self.client_secret,
#                                           oauth2 = True,
#                                           refresh_cb=self.__refresh_cb,
#                                           access_token=token['access_token'],
#                                           refresh_token=token['refresh_token'],
#                                           expires_at=expires_at,
#                                           redirect_uri=redirect_uri,
#                                           system=system,
#                                           **kwargs)
#         self.client.refresh_token()
#         self.__running = True
#         self.token_daemon = FitbitTokenDaemon(self)
#         self.token_daemon.start()
    
#     @property
#     def running(self):
#         return self.__running
    
#     @running.setter
#     def running(self, val:bool):
#         if not isinstance(val, bool):
#             raise TypeError('running must be a boolean')
#         self.__running = val
    
#     def stop(self):
#         self.running = False
    
#     # Public Methods
#     def _assertions(self,dtype:str,begin_date:dt.datetime,end_date:dt.datetime,resource:str,**kwargs):
#         '''
#         Function to assert the input parameters for the time series data functions.
#         '''
#         max_days = 30 if (dtype == 'activities' and resource == 'activityCalories') else 1095 

#         assert dtype in self.TS_RESOURCE.keys(), "dtype not recognized."
#         assert resource in self.TS_RESOURCE[dtype], "Resource not recognized."
#         assert (begin_date or kwargs.get('period',None)), "Need to pass a begin_date or a period."
#         assert not (begin_date and kwargs.get('period',None)), "Cannot pass both begin_date and period."
        
#         if kwargs.get('period',None):
#             assert type(kwargs['period']) == int, "Period must be an integer."
#             assert kwargs['period'] > 0 and kwargs['period'] < max_days, f"Period must be between 1 and {max_days} days."
#         if begin_date and end_date:
#             assert begin_date < end_date, "Begin date must be before end date."
        

#     def body_time_series(self,begin_date:dt.datetime = None,end_date:dt.datetime = None,resource:str=None,**kwargs):
#         '''
#         Function to get the body time series data.

#         Parameters:
#         begin_date:dt.datetime: The date from which to start the data retrieval. If None, period must be passed.
#         end_date:dt.datetime: The date to end the data retrieval. Default is today.
#         resource:str: The resource to get the data for. Default is weight. Options are weight, fat, bmi.
        
#         **kwargs: 
#         period = The period to get the data for. Must be an integer representing the number of days.

#         '''
#         dtype = 'body'
#         url = '{0}/{1}/user/-/body/{resource}/date/{begin_date}/{end_date}.json'
#         resource = resource or 'weight' #default to weight

#         self._assertions(dtype,begin_date,end_date,resource,**kwargs)
        
#         if not end_date:
#             end_date = dt.datetime.now()
#         if not begin_date:
#             begin_date = end_date - dt.timedelta(days=kwargs['period'])
#         begin_date = begin_date.strftime('%Y-%m-%d')
#         end_date = end_date.strftime('%Y-%m-%d')

#         payload = dict(resource=resource,begin_date=begin_date,end_date=end_date)
#         url = url.format(*self._get_common_args(),**payload)

#         return self.make_request(url)[f'{dtype}-{resource}']
    
#     def activity_time_series(self,begin_date:dt.datetime = None,end_date:dt.datetime = None,resource:str=None,**kwargs):
#         '''
#         Function to get the activity time series data.

#         Parameters:
#         begin_date:dt.datetime: The date from which to start the data retrieval. If None, period must be passed.
#         end_date:dt.datetime: The date to end the data retrieval. Default is today.
#         resource:str: The resource to get the data for. Default is steps. Options are activityCalories, calories, caloriesBMR, distance, floors, minutesSedentary, minutesLightlyActive, minutesFairlyActive, minutesVeryActive, steps.

#         **kwargs:
#         period = The period to get the data for. Must be an integer representing the number of days.
#         '''
#         dtype = 'activities'
#         url = '{0}/{1}/user/-/activities/{resource}/date/{begin_date}/{end_date}.json'
#         resource = resource or 'steps' #default to weight

#         self._assertions(dtype,begin_date,end_date,resource,**kwargs)

#         if not end_date:
#             end_date = dt.datetime.now()
#         if not begin_date:
#             begin_date = end_date - dt.timedelta(days=kwargs['period'])
#         begin_date = begin_date.strftime('%Y-%m-%d')
#         end_date = end_date.strftime('%Y-%m-%d')

#         payload = dict(resource=resource,begin_date=begin_date,end_date=end_date)
#         url = url.format(*self._get_common_args(),**payload)

#         return self.make_request(url)[f'{dtype}-{resource}']
    
#     def food_time_series(self,begin_date:dt.datetime = None,end_date:dt.datetime = None,resource:str=None,**kwargs):
#         dtype = 'foods'
#         url = '{0}/{1}/user/-/foods/log/{resource}/date/{begin_date}/{end_date}.json'
#         resource = resource or 'caloriesIn'

#         self._assertions(dtype,begin_date,end_date,resource,**kwargs)
        
#         if not end_date:
#             end_date = dt.datetime.now()
#         if not begin_date:
#             begin_date = end_date - dt.timedelta(days=kwargs['period'])
#         begin_date = begin_date.strftime('%Y-%m-%d')
#         end_date = end_date.strftime('%Y-%m-%d')

#         payload = dict(resource=resource,begin_date=begin_date,end_date=end_date)
#         url = url.format(*self._get_common_args(),**payload)

#         return self.make_request(url)
    
#     def get_macros_by_date(self,date:dt.datetime)->dict:
#         '''
#         Function to get the macronutrient data for a given date.
#         '''
#         url = '{0}/{1}/user/-/foods/log/date/{date}.json'
#         date = date.strftime('%Y-%m-%d')
#         url = url.format(*self._get_common_args(),date=date)
#         return self.make_request(url)['summary']
    
#     def macros_time_series(self,begin_date:dt.datetime=None,end_date:dt.datetime=None,**kwargs)->list[dict]:
#         '''
#         Function to get the macronutrient data for a given date range.
#         '''
#         self._assertions('foods',begin_date,end_date,'caloriesIn',**kwargs)
#         if not end_date:
#             end_date = dt.datetime.now()
#         if not begin_date:
#             begin_date = end_date - dt.timedelta(days=kwargs['period'])
#         res = []
#         while begin_date <= end_date:
#             _dict = self.get_macros_by_date(begin_date)
#             _dict['dateTime'] = begin_date.strftime('%Y-%m-%d')
#             res.append(_dict)
#             begin_date += dt.timedelta(days=1)
#         return res
    
#     def heart_time_series(self,begin_date:dt.datetime = None,end_date:dt.datetime = None,**kwargs):
#         url = '{0}/{1}/user/-/activities/heart/date/{begin_date}/{end_date}.json'
        
#         assert begin_date or kwargs.get('period',None), "Need to pass a begin_date or a period."
#         assert not (begin_date and kwargs.get('period',None)), "Cannot pass both begin_date and period."
#         if kwargs.get('period',None):
#             assert type(kwargs['period']) == int, "Period must be an integer."
#             assert kwargs['period'] > 0 and kwargs['period'] < 364, "Period must be between 1 and 364 days."
        
#         if not end_date:
#             end_date = dt.datetime.now()
#         if not begin_date:
#             begin_date = end_date - dt.timedelta(days=kwargs['period'])
#         begin_date = begin_date.strftime('%Y-%m-%d')
#         end_date = end_date.strftime('%Y-%m-%d')
        
#         payload = dict(begin_date=begin_date,end_date=end_date)
#         url = url.format(*self._get_common_args(),**payload)

#         return self.make_request(url)['activities-heart']

#     def sleep_time_series(self,begin_date:dt.datetime = None,end_date:dt.datetime = None,**kwargs):
#         url = '{0}/{1}/user/-/sleep/date/{begin_date}/{end_date}.json'
        
#         assert begin_date or kwargs.get('period',None), "Need to pass a begin_date or a period."
#         assert not (begin_date and kwargs.get('period',None)), "Cannot pass both begin_date and period."
#         if kwargs.get('period',None):
#             assert type(kwargs['period']) == int, "Period must be an integer."
#             assert kwargs['period'] > 0 and kwargs['period'] < 100, "Period must be between 1 and 100 days."
        
#         if not end_date:
#             end_date = dt.datetime.now()
#         if not begin_date:
#             begin_date = end_date - dt.timedelta(days=kwargs['period'])
#         begin_date = begin_date.strftime('%Y-%m-%d')
#         end_date = end_date.strftime('%Y-%m-%d')
        
#         payload = dict(begin_date=begin_date,end_date=end_date)
#         url = url.format(*self._get_common_args(),**payload)

#         return self.make_request(url)['sleep']
#     # Private methods 
#     @staticmethod
#     def __refresh_cb(token:dict)->None:
#         '''
#         Function to be passed to the FITBIT object to store the new token. If not passed, the app needs to be re-authed every time the script runs. The Access Token only lasts for an hour.
#         '''
#         assert [key in token for key in ['access_token', 'refresh_token', 'expires_at']], "Token is missing a key"
#         with open(TOKEN_PATH, 'wb') as f:
#             pickle.dump(token, f)

#     @staticmethod
#     def __get_creds()->tuple[str,str]:
#         with open(CREDS_PATH, 'r') as f:
#             creds = json.load(f)
#         return creds['CLIENT_ID'], creds['CLIENT_SECRET']
    
#     def __get_token(self)->dict:
#         try:
#             with open(TOKEN_PATH, 'rb') as f:
#                 token = pickle.load(f)
#             if token['expires_at'] < time.time():
#                 token = self.__authenticate()
#         except FileNotFoundError:
#             token = self.__authenticate()
#         except:
#             raise
#         return token

#     def __authenticate(self)->dict:
#         server = gather_keys_oauth2.OAuth2Server(self.client_id, self.client_secret)
#         server.browser_authorize()
#         token = server.fitbit.client.session.token
#         self.__refresh_cb(token)
#         return token

#     def __del__(self):
#         self.running = False
#         self.token_daemon.stop()
#         self.token_daemon.join()
#         sleep(self.token_daemon.sleep_interval*2)
#         super(FitbitServer, self).__del__()

# def raise_or_return(df:pd.DataFrame,left_length:int,right_length:int)->pd.DataFrame:
#     '''
#     check if _merge column is not both, if so raise an error, otherwise return the dataframe
#     '''
#     if df['_merge'].value_counts()['both'] != len(df):
#         raise ValueError('DataFrames did not merge correctly.')
#     if left_length != right_length:
#         raise ValueError('DataFrames are not the same length.')
#     if left_length != len(df):
#         raise ValueError('DataFrames did not merge correctly.')
#     return df.drop(columns=['_merge'])

# class FitbitDataDaemon:
#     def __init__(self):
#         self.fs = FitbitServer()
#         self.__connect()
#         self.requestsRemaining = RATE_LIMIT
    
#     def stop(self):
#         self.conn.close()
#         self.fs.stop()        

#     def __connect(self):
#         self.conn = sqlite3.connect(DB_PATH)
#         self.conn.cursor().execute('''CREATE TABLE IF NOT EXISTS fitbit (scrapeTime TEXT, 
#                                                                         dateTime TEXT,
#                                                                         steps INTEGER,
#                                                                         weight REAL,
#                                                                         calories INTEGER,
#                                                                         carbs INTEGER,
#                                                                         fat INTEGER,
#                                                                         protein INTEGER,
#                                                                         sodium INTEGER)''')
#         self.conn.commit()
        
#     def getData(self)->pd.DataFrame:
#         steps:list[dict] = self.fs.activity_time_series(period=DAYS_TO_UPDATE) # dict(dateTime, value)
#         weight:list[dict] = self.fs.body_time_series(period=DAYS_TO_UPDATE) # dict(dateTime, value)
#         macros:list[dict] = self.fs.macros_time_series(period=DAYS_TO_UPDATE) # dict(dateTime, calories, carbs, fat, fiber, protein, sodium)

#         df = pd.DataFrame(steps)
#         df = pd.merge(df,pd.DataFrame(weight),on='dateTime',how='outer',suffixes=('_steps','_weight'), indicator=True).pipe(raise_or_return,len(steps),len(weight))
#         df = pd.merge(df,pd.DataFrame(macros),on='dateTime',how='outer',suffixes=('','_macros'),indicator=True).pipe(raise_or_return,len(df),len(macros))
#         df.columns = df.columns.str.replace('value_','')

#         df.steps = df.steps.fillna(0).astype(int)
#         df.weight = df.weight.fillna(0).astype(float)
#         df.calories = df.calories.fillna(0).astype(int)
#         df.carbs = df.carbs.fillna(0).astype(int)
#         df.fat = df.fat.fillna(0).astype(int)
#         df.protein = df.protein.fillna(0).astype(int)
#         df.sodium = df.sodium.fillna(0).astype(int)
#         df = df['dateTime steps weight calories carbs fat protein sodium'.split()]

#         self.requestsRemaining -= NUM_REQUESTS_PER_UPDATE
#         return df
    
#     def getUpdatedData(self)->pd.DataFrame:
#         '''
#         From the database, select the most recent scrapeTime for each dateTime. Then, merge the new data with the database data and calculate the difference between the two.
#         Only return the data that has changed.
#         '''
#         db = pd.read_sql('''
#                             SELECT fitbit.*
#                             FROM fitbit
#                             JOIN (
#                                 SELECT dateTime, MAX(scrapeTime) AS maxScrapeTime
#                                 FROM fitbit
#                                 GROUP BY dateTime
#                             ) grouped
#                             ON fitbit.dateTime = grouped.dateTime AND fitbit.scrapeTime = grouped.maxScrapeTime
#                             ''', self.conn)
        
#         updatedData = (self.getData().merge(db, on='dateTime', how='left', suffixes=('_new','_old'))
#             .fillna(0)
#             .drop(columns=['scrapeTime'])
#             .assign(steps = lambda x: x.steps_new - x.steps_old,
#                     weight = lambda x: x.weight_new - x.weight_old,
#                     calories = lambda x: x.calories_new - x.calories_old,
#                     carbs = lambda x: x.carbs_new - x.carbs_old,
#                     fat = lambda x: x.fat_new - x.fat_old,
#                     protein = lambda x: x.protein_new - x.protein_old,
#                     sodium = lambda x: x.sodium_new - x.sodium_old)
#             .loc[lambda self: (self['steps weight calories carbs fat protein sodium'.split()].abs()>0).any(axis=1)]
#             [['dateTime','steps_new','weight_new','calories_new','carbs_new','fat_new','protein_new','sodium_new']]
#             .rename(columns={'steps_new':'steps',
#                             'weight_new':'weight',
#                             'calories_new':'calories',
#                             'carbs_new':'carbs',
#                             'fat_new':'fat',
#                             'protein_new':'protein',
#                             'sodium_new':'sodium'})
#             .assign(scrapeTime = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
#         return updatedData
    
#     def updateDatabase(self)->str:
#         data = self.getUpdatedData()
#         if not data.empty:
#             data.to_sql('fitbit', self.conn, if_exists='append', index=False)
#             return "FitbitDataDaemon saved to database. "
#         return "FitbitDataDaemon no new data to save. "
    
#     def main(self):
#         while True:
#             sleepTime = SLEEP_TIME
#             report = "[{}] ".format(dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
#             try:
#                 report += self.updateDatabase() + f"Requests remaining: {self.requestsRemaining}. "
#             except exceptions.HTTPTooManyRequests:
#                 report += "FitbitDataDaemon encountered a rate limit error. Sleeping until top of the hour. "
#                 now = dt.datetime.now()
#                 topOfHour = now.replace(minute=0, second=0, microsecond=0)+dt.timedelta(hours=1)
#                 # sleep until one minute past the hour as the reset time is not exactly on the hour.
#                 # This prevents the script from hitting the rate limit again at the top of the hour
#                 # and then sleeping for another hour.
#                 sleepTime = (topOfHour-now).total_seconds()+60 
#             except KeyboardInterrupt:
#                 report += "FitbitDataDaemon was interrupted. "
#                 sleepTime = 0
#                 self.stop()
#                 break
#             except Exception as e:
#                 report += f"FitbitDataDaemon encountered an error: {e}. "
#             finally:
#                 report += "[sleeping for {}s]".format(sleepTime)
#                 print(report)
#                 sleep(sleepTime)

# fdd = FitbitDataDaemon()
    

In [ ]:
import sys
sys.path.append('../..')
# from DashBoard.src import FitbitDataDaemon
# fdd = FitbitDataDaemon()

In [ ]:
# fdd.main()

In [ ]:
from DashBoard.src import main

In [ ]:
main()

In [ ]:
1